<a href="https://colab.research.google.com/github/Oleksandr190378/data-computing/blob/main/advanced_rag_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q langchain sentence-transformers cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.1/233.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7

In [3]:
!pip install rank_bm25

In [ ]:
!pip install langchain-community


In [12]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 4.8 MB/s eta 0:00:00


In [17]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 37.3 MB/s eta 0:00:00


In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.vectorstores import FAISS
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.document_loaders import PyPDFLoader

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import os
from getpass import getpass

HF_token = getpass("Enter your hugging face API token:")
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_token

Enter your hugging face API token:··········


In [13]:
dataset_folder_path='/content/drive/MyDrive/Colab Notebooks/healthy_food/'
documents=[]
for file in os.listdir(dataset_folder_path):
  loader = PyPDFLoader(dataset_folder_path+file)
  documents.extend(loader.load())
documents[:3]

[Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/healthy_food/Tarilka.pdf', 'page': 0}, page_content='1 \n УКРАЇНСЬКА ТАРІЛКА ЗДОРОВОГО ХАРЧУВАННЯ  \nОльга Ільюхіна  \n Нашим найпершим інструментом у навчанні є Тарілка здорового \nхарчування. Саме про неї ми з вами поговоримо цього тижня.  \nНасправді Тарілка здорового харчування – це зручна форма візуалізації \nнашого щоденного раціону.  В основі Тарілки лежать Національні \nрекомендації щодо здорового харчування дорослих, які були розроблені \nнаціональною асоціацією дієтологів  та затверджені Міністерством охорони \nздоров ’я України наприкінці 2017 року. Ви зможете ці рекомендації  \nзавантажити і детально ознайомитися з повним документом  у додаткових \nматеріалах до цієї теми.  \nЯкщо ви ознайомитеся з рекомендаціями відомих міжнародних \nорганізацій , таких як Всесвітня організація охорони здоров’я  (ВООЗ ) або \nподивитеся на національні рекомендації інших країн, то побачите багато \nспільного з нашою, укра

In [14]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=512,chunk_overlap=50)
text_splits=text_splitter.split_documents(documents)
print(len(text_splits))

169


In [15]:
embeddings=HuggingFaceInferenceAPIEmbeddings(
    api_key=HF_token,
    model_name='BAAI/bge-base-en-v1.5'
)

In [18]:
vectorstore = FAISS.from_documents(text_splits, embeddings)

In [19]:
retriever_vectordb = vectorstore.as_retriever(search_kwargs={"k": 5})

In [20]:
keyword_retriever = BM25Retriever.from_documents(text_splits)
keyword_retriever.k =  5

In [21]:
ensemble_retriever = EnsembleRetriever(retrievers=[retriever_vectordb,keyword_retriever],
                                       weights=[0.5, 0.5])

In [22]:
query = 'вплив цукру на наш організм'

In [23]:
docs_rel=ensemble_retriever.get_relevant_documents(query)
docs_rel

<ipython-input-23-c84b9c752c34>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs_rel=ensemble_retriever.get_relevant_documents(query)


[Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/healthy_food/Recommendor.pdf', 'page': 1}, page_content='харчові продукти, що пройшли технологічну обробку. Додавання'),
 Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/healthy_food/Recommendor.pdf', 'page': 12}, page_content='13 \n - Як побудувати оптимальний режим харчування та адаптувати його до \nщоденного графіку Вашого життя? Що таке здорові харчові звички і \nчому важлива здорова  харчова поведінка?  \n \nЧастота та регулярність прийому їжі є важливим компонентом харчової \nповедінки та має досить виражений вплив на процеси травлення, обміну \nречовин, сигнальні зв’язки між центральною нервовою системою, \nорганами травлення та ендокринною системою : \n \no Оптимальний режим харчування позитивно впливає на фізіологію'),
 Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/healthy_food/water.pdf', 'page': 2}, page_content='Такі напої сприяють розвитку передчасного старіння

In [24]:
from langchain.llms import HuggingFaceHub
model=HuggingFaceHub(repo_id='HuggingFaceH4/zephyr-7b-alpha',
                     model_kwargs={"temperature":0.5,"max_new_tokens":512,"max_length":64})

<ipython-input-24-fdeebd7bc6fd>:2: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  model=HuggingFaceHub(repo_id='HuggingFaceH4/zephyr-7b-alpha',


In [25]:
Cohere_API_token = getpass()
os.environ["COHERE_API_KEY"] =Cohere_API_token

··········


In [26]:
compressor = CohereRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=ensemble_retriever
)
compressed_docs = compression_retriever.get_relevant_documents(query)

<ipython-input-26-fed6bf08a6bd>:1: LangChainDeprecationWarning: The class `CohereRerank` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import CohereRerank``.
  compressor = CohereRerank()


In [27]:
compression_retriever

ContextualCompressionRetriever(base_compressor=CohereRerank(client=<cohere.client.Client object at 0x7e8b3aac3220>, top_n=3, model='rerank-english-v2.0', cohere_api_key=None, user_agent='langchain'), base_retriever=EnsembleRetriever(retrievers=[VectorStoreRetriever(tags=['FAISS', 'HuggingFaceInferenceAPIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7e8b12c54ac0>, search_kwargs={'k': 5}), BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x7e8b12c54640>, k=5)], weights=[0.5, 0.5]))

In [28]:
compressed_docs

[Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/healthy_food/unhealthy_food.pdf', 'page': 0, 'relevance_score': 0.94891727}, page_content='продукти  привабливими та смачними саме завдяки використ анню  велик ої \nкільк ості цукру, солі та жиру. І таких продуктів ми можемо спожити набагато \nбільше, ніж варто .  \n \nЦукор  \nЩо відбувається з організмом при тривалому без контрольному \nспоживанні солод кого?  Це призводить до суттєвого порушення енергетичного \nбалансу , ми отримуємо більше енергії, ніж використовуємо . Людина починає \nнабирати зайву вагу та може отрим ати неприємні  наслідки  у вигляді карієсу,'),
 Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/healthy_food/unhealthy_food.pdf', 'page': 1, 'relevance_score': 0.9280184}, page_content="До вільного цукру належать  фруктоза , глюкоза та сахароза.  \nСкільки можна їсти цукру? І чи варто зовсім відмовлятися від нього ? Чи є \nмед кращою альтернативою ?  \n Сучасні рекомендації Все

In [47]:
template = """
<|system|>>
You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers in the language of request. Please tell 'I don't know' if user query is not in CONTEXT

CONTEXT: {context}
</s>
<|user|>
{query}
</s>
<|assistant|>
"""

In [48]:
prompt = ChatPromptTemplate.from_template(template)

In [49]:
output_parser = StrOutputParser()

In [50]:
chain = (
    {"context": compression_retriever, "query": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

In [51]:
query = 'чи шкодить сіль організму людини?'

In [52]:
response = chain.invoke(query)

In [55]:
from transformers import pipeline

In [56]:
translator = pipeline("translation_en_to_uk", model="Helsinki-NLP/opus-mt-en-uk")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/305M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/809k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.01M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [57]:
import re
cleaned_response = re.sub(r'<\|system\|>.*?<\|assistant\|>', '', response, flags=re.DOTALL)
cleaned_response

"Human: \n\nExcessive intake of salt (sodium) can increase the risk of hypertension (high blood pressure), which is a major risk factor for heart disease and stroke. So, it is advisable to limit the consumption of salt and choose low-sodium options when possible. However, a small amount of salt is essential for the body's proper functioning, and a healthy adult needs only about 500 milligrams (mg) per day."

In [58]:
translated_response = translator(cleaned_response)[0]['translation_text']
translated_response

"Людина: надмірне споживання солі (содіум) може збільшити ризик гіпертонії (високий кров'яний тиск), який є основним чинником ризику серцевих захворювань та інсульту. Отже, слід обмежити споживання солі і при можливості вибрати знежирення. Але для належної функціонування організму потрібна невелика кількість солі, а здоровому дорослому потрібно лише близько 500 міліграмів (мг) на день."